- Installing mlflow
- Starting a local mlflow tracking server
- Logging and registering a model with mlflow
- Loading a logged model for inference using pyfunc flavor
- Viewing the experiment results in MLflow ui

In [4]:
import mlflow
from mlflow.models import infer_signature
import pandas  as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
#Set the tracking uri
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [6]:
#Load teh dataset
X,y =datasets.load_iris(return_X_y=True)

In [7]:
#Split teh data into train and test 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)

In [8]:
params={'penalty':'l2' , 'solver':'lbfgs','max_iter':1000,'multi_class':'auto' , 'random_state':8888}

#Defining the model
lr=LogisticRegression(**params)
lr.fit(X_train , y_train)


c:\Users\HP\Desktop\MLFlow\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [9]:
y_pred=lr.predict(X_test)
y_pred

array([1, 0, 2, 0, 2, 1, 2, 0, 2, 1, 1, 0, 0, 0, 2, 0, 1, 0, 1, 1, 0, 1,
       2, 1, 2, 0, 2, 2, 2, 0])

In [10]:
acc=accuracy_score(y_test , y_pred)
print(acc)

0.9666666666666667


In [11]:
#mlflow tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")

#Create a new MLFLOW experiment
mlflow.set_experiment("MLFLOW Quickstart")

#Start the MLflow run
with mlflow.start_run():
    mlflow.log_params(params) #log the parameters
    mlflow.log_metric("accuracy" , acc) #log the accuracy
    mlflow.set_tag("Training info" , "Basic LR model for iris data") #log the tag
    signature=infer_signature(X_train , lr.predict(X_test)) #log the signature
    #Log th emodel
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart"
    )




Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/04/22 10:42:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 3
Created version '3' of model 'tracking-quickstart'.


🏃 View run whimsical-mule-962 at: http://127.0.0.1:5000/#/experiments/882138542450818522/runs/dcf1174e847246c0bc70f3b3027428ce
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/882138542450818522


### INferenecing and validating model

In [17]:
loaded_model = mlflow.pyfunc.load_model(model_uri)
predictions = loaded_model.predict(X_test)

iris_features_name=datasets.load_iris().feature_names

result=pd.DataFrame(X_test, columns=iris_features_name)
result['actual_class']=y_test
result['predicted_class']=predictions

In [18]:
result

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.7,3.1,4.7,1.5,1,1
1,5.5,4.2,1.4,0.2,0,0
2,6.7,2.5,5.8,1.8,2,2
3,5.0,3.5,1.6,0.6,0,0
4,6.9,3.1,5.1,2.3,2,2
5,5.6,2.9,3.6,1.3,1,1
6,6.0,2.7,5.1,1.6,1,2
7,5.0,3.6,1.4,0.2,0,0
8,6.5,3.2,5.1,2.0,2,2
9,5.9,3.0,4.2,1.5,1,1


### Model Registry